**Buffer Water Flowlines**

In [ ]:
import geopandas as gpd
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root))

from scripts import config

Let's inspect the data.

In [ ]:
# Load dataset
gdf = gpd.read_file(config.GDB_PATH, layer="Water_Flow_CO")

print(gdf.columns.tolist())
gdf.head()


First of all, let's drop all unnecessary columns.

In [ ]:
gdf_filtered = gdf[["geometry", "permanent_Identifier", "flowdir", "Shape_Length"]]
gdf_filtered.head()

For our purposes, we don't really need elevation in the geometry (multilinestring Z); if we really need elevation, we can get that from a separate elevation layer. The Z coordinates might be messing with gpkg exporting, so let's drop it.

In [ ]:
from shapely.geometry import LineString, MultiLineString

def drop_z(geom):
    if geom is None:
        return None
    if geom.has_z:
        if isinstance(geom, LineString):
            return LineString([(pt[0], pt[1]) for pt in geom.coords])
        elif isinstance(geom, MultiLineString):
            return MultiLineString([
                LineString([(pt[0], pt[1]) for pt in line.coords])
                for line in geom.geoms
            ])
    return geom

gdf_drop_z = gdf_filtered.copy()
gdf_drop_z["geometry"] = gdf_filtered["geometry"].apply(drop_z)
gdf_drop_z.head()

Let's do some cleaning. First, we need to project to the right CRS, then check for empty/invalid geometries. There's also a weird UUID entry in permanent_Identifier - let's cast that field to string just in case. Finally, let's rebuffer the geometry and simplify with a tolerance of 1 to ensure exporting goes smoothly.

In [ ]:
print("Initial count:", len(gdf))

# Remove invalid or empty geometries
gdf_drop_z = gdf_drop_z[~gdf.geometry.is_empty & gdf.geometry.is_valid]
print("After validity check:", len(gdf))

gdf_drop_z["permanent_Identifier"] = gdf_drop_z["permanent_Identifier"].astype(str)


**Project to CRS and Buffer**

First, let's project to a common CRS.

In [ ]:
gdf_projected = gdf_drop_z.to_crs(config.BUFFER_CRS)


Now, let's buffer by the distance specified in config.py.

In [ ]:
gdf_buffered = gdf_projected.copy()
gdf_buffered = gdf_buffered.set_geometry(gdf_buffered.geometry.buffer(config.BUFFER_DISTANCE_WATER_METERS))
gdf_buffered.head()

Let's convert all polygons to multipolygons to ensure consistent typing.

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

def ensure_multipolygon(geom):
    if geom is None or geom.is_empty:
        return geom
    if isinstance(geom, Polygon):
        return MultiPolygon([geom])
    elif isinstance(geom, MultiPolygon):
        return geom
    return geom  # In case something unexpected sneaks in

gdf_multipoly = gdf_buffered.copy()
gdf_multipoly["geometry"] = gdf_multipoly["geometry"].apply(ensure_multipolygon)

Now let's export the data.

In [ ]:
output_folder = config.OUTPUT_DIR / "water_flowlines_buffered"
output_folder.mkdir(parents=True, exist_ok=True)

output_fp = output_folder / "water_flowlines_buffered.gpkg"
gdf_multipoly.to_file(output_fp, driver="GPKG")

print("Saved buffered flowlines to:", output_fp)